In [1]:
def parse_results_num(file):
    with open(file) as f:
        lines = f.readlines() 
        num_covered_targets = 0
        num_potential_faults = 0
        num_potential_500_faults = 0
        
        # if version == 'v200':
        #     pt500_text_to_check = "* Potential 500 faults IDs: "
        # else:
        #     pt500_text_to_check = "* Potential 500 faults: "

        for line in lines:
            if "* Covered targets (lines" in line:
                num_covered_targets = int(line.split("etc.): ")[1].split('\n')[0])
            if "* Potential faults: " in line:
                num_potential_faults = int(line.split('faults: ')[1].split('\n')[0])
            if "* Potential 500 faults: " in line:
                num_potential_500_faults = int(line.split("* Potential 500 faults: ")[1].split('\n')[0])

        if num_covered_targets == 0 and num_potential_faults == 0 and num_potential_500_faults == 0:
            print(f"Results missing for file: {file}")
        
        return num_covered_targets, num_potential_faults, num_potential_500_faults
       


In [61]:
num_iterations = 11
app = "languagetool"
algorithms = ['MOSA', 'MISH']#, 'MISHMOSA']
fitness_functions = ['lower_median', 'weighted_size']
# if algorithm == 'MOSA':
result_format = f'with_500_faults_EM_logs'
version = 'v200'
fitness_abbrev = {'lower_median': 'LM', 'weighted_size': 'WS'}
# else:
    # result_format = f'with_500_faults_{fitness_function}_EM_logs'

all_runs_covered_targets = dict()
all_runs_potential_faults = dict()
all_runs_potential_500_faults = dict()

for algorithm in algorithms:
    for fitness_function in fitness_functions:
        
        if algorithm == 'MOSA' and fitness_function == 'weighted_size':
            continue

        covered_targets = []
        potential_faults = []
        potential_500_faults = []
        for i in range(1, num_iterations):
            file = f"../../results_{app}_{version}_{fitness_function}_better_sampling_latest/{app}_{result_format}_{algorithm}_{i}.txt"
            ct, pt, pt500 = parse_results_num(file)
            covered_targets.append(ct)
            potential_faults.append(pt)
            potential_500_faults.append(pt500)
        
        if algorithm == 'MOSA':
            all_runs_covered_targets[algorithm] = covered_targets
            all_runs_potential_faults[algorithm] = potential_faults
            all_runs_potential_500_faults[algorithm] = potential_500_faults
        
        else:
            all_runs_covered_targets[algorithm + " - " + fitness_abbrev[fitness_function]] = covered_targets
            all_runs_potential_faults[algorithm + " - " + fitness_abbrev[fitness_function]] = potential_faults
            all_runs_potential_500_faults[algorithm + " - " + fitness_abbrev[fitness_function]] = potential_500_faults



In [62]:
MOSA_covered_targets = all_runs_covered_targets['MOSA']
MISH_LM_covered_targets = all_runs_covered_targets['MISH - LM']
MISH_WS_covered_targets = all_runs_covered_targets['MISH - WS']

# Print them in table format
# print("MOSA \t MISH - LM \t MISH - WS")
for i in range(num_iterations - 1):
    # print(f"{MOSA_covered_targets[i]} \t {MISH_covered_targets[i]} \t {MISHMOSA_covered_targets[i]}")
    print(f"{MOSA_covered_targets[i]} \t {MISH_LM_covered_targets[i]} \t {MISH_WS_covered_targets[i]}")

16306 	 2911 	 2888
3012 	 2916 	 2932
3058 	 3180 	 3170
3084 	 2904 	 10658
3104 	 2899 	 2977
2950 	 2926 	 2901
3071 	 3026 	 3055
3157 	 3033 	 2906
18473 	 3029 	 3104
14278 	 2919 	 2969


In [63]:
MOSA_covered_targets = all_runs_potential_faults['MOSA']
MISH_LM_covered_targets = all_runs_potential_faults['MISH - LM']
MISH_WS_covered_targets = all_runs_potential_faults['MISH - WS']

# Print them in table format
# print("MOSA \t MISH - LM \t MISH - WS")
for i in range(num_iterations - 1):
    # print(f"{MOSA_covered_targets[i]} \t {MISH_covered_targets[i]} \t {MISHMOSA_covered_targets[i]}")
    print(f"{MOSA_covered_targets[i]} \t {MISH_LM_covered_targets[i]} \t {MISH_WS_covered_targets[i]}")

6 	 7 	 5
6 	 7 	 6
7 	 6 	 6
5 	 8 	 7
7 	 6 	 6
7 	 6 	 6
7 	 7 	 7
7 	 7 	 6
8 	 6 	 7
8 	 6 	 7


In [64]:
MOSA_covered_targets = all_runs_potential_500_faults['MOSA']
MISH_LM_covered_targets = all_runs_potential_500_faults['MISH - LM']
MISH_WS_covered_targets = all_runs_potential_500_faults['MISH - WS']

# Print them in table format
# print("MOSA \t MISH - LM \t MISH - WS")
for i in range(num_iterations - 1):
    # print(f"{MOSA_covered_targets[i]} \t {MISH_covered_targets[i]} \t {MISHMOSA_covered_targets[i]}")
    print(f"{MOSA_covered_targets[i]} \t {MISH_LM_covered_targets[i]} \t {MISH_WS_covered_targets[i]}")

3 	 5 	 3
4 	 5 	 4
5 	 4 	 4
3 	 6 	 4
5 	 4 	 4
5 	 4 	 4
5 	 5 	 5
5 	 5 	 4
5 	 4 	 5
5 	 4 	 5


# Computing statistics for paper.

In [6]:
from statistics import median
import numpy as np
num_iterations = 11
applications = ['catwatch', 'features-service', 'scout-api', 'proxyprint', 'languagetool']
algorithms = ['MOSA', 'MISH']#, 'MISHMOSA']
fitness_functions = ['lower_median', 'weighted_size']
abbrev_map = {'lower_median': 'LM', 'weighted_size': 'WS'}
result_format = f'with_500_faults_EM_logs'
version = 'v320'

app_results = dict()
for app in applications:
    app_results[app] = dict()
    all_runs_covered_targets = dict()
    all_runs_potential_faults = dict()
    all_runs_potential_500_faults = dict()

    for fitness_function in fitness_functions:
        for algorithm in algorithms:
            if algorithm == 'MOSA' and fitness_function == 'weighted_size':
                continue

            covered_targets = []
            potential_faults = []
            potential_500_faults = []
            for i in range(1, num_iterations):
                file = f"../../results_{app}_{version}_{fitness_function}_latest/{app}_{result_format}_{algorithm}_{i}.txt"
                ct, pt, pt500 = parse_results_num(file, version)
                covered_targets.append(ct)
                potential_faults.append(pt)
                potential_500_faults.append(pt500)
            
            if algorithm == 'MOSA':
                method_name = algorithm
            else:
                method_name = f"{algorithm} - {abbrev_map[fitness_function]}"

            all_runs_covered_targets[method_name] = covered_targets
            all_runs_potential_faults[method_name] = potential_faults
            all_runs_potential_500_faults[method_name] = potential_500_faults

    app_results[app]['covered_targets'] = all_runs_covered_targets
    app_results[app]['potential_faults'] = all_runs_potential_faults
    app_results[app]['potential_500_faults'] = all_runs_potential_500_faults

# first print the method names in one line
method_names = ['MOSA', 'MISH - LM', 'MISH - WS']#, 'MISHMOSA - LM', 'MISHMOSA - WS']
print('\t'.join(method_names))

# Then print the median covered targets
output_to_print = []

metric = 'potential_500_faults'

for app in applications:
    app_output = []
    for method_name in method_names:
        method_results = app_results[app][metric][method_name]
        med_res = str(median(method_results))
        q75, q25 = np.percentile(method_results, [75 ,25])
        iqr = q75 - q25
        app_output.append(med_res)
        app_output.append(str(iqr))


    output_to_print.append(app_output)

for output in output_to_print:
    print('\t'.join(output))






MOSA	MISH - LM	MISH - WS
10.0	2.0	7.0	1.0	7.0	1.75
25.0	0.75	24.0	0.75	23.5	1.75
57.5	2.75	52.5	2.5	45.0	4.0
29.0	4.75	34.0	2.0	34.5	2.0
5.0	0.75	5.5	1.0	4.0	0.75
